In [51]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

# %matplotlib inline

### Creating a model class which inherits the nn.Module

which takes 4 inputs as paramters
- sepal length
- sepal width
- petal length
- petal width

forward it to a hidden layer 1 (H1) with n number of neurons >> \
which sends it to hidden layer 2 (H2) with the same n number of neurons >> \
then classifies as 3 classes of flowers



In [66]:
class Model(nn.Module):
    def __init__(self, in_feature = 4, h1 = 7, h2 = 7, out_features = 3):
        super().__init__()
        self.fc1 = nn.Linear(in_feature, h1)
        self.fc2 = nn.Linear(h1, h2)
        self.out = nn.Linear(h2, out_features)

    def forward(self, x):
        x = F.relu(self.fc1(x)) #rectified linear unit function element-wise
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x

In [68]:
torch.manual_seed(33) #randomizing the initiation

model = Model() #creating an instance of the model class

In [55]:
#creating dataframe of the iris dataset

df = pd.read_csv('iris-dataset/iris.data', header = None, names = ['sepal_length' , 'sepal_width', 'petal_length', 'petal_width', 'species'])
df.head(10)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
5,5.4,3.9,1.7,0.4,Iris-setosa
6,4.6,3.4,1.4,0.3,Iris-setosa
7,5.0,3.4,1.5,0.2,Iris-setosa
8,4.4,2.9,1.4,0.2,Iris-setosa
9,4.9,3.1,1.5,0.1,Iris-setosa


In [56]:
df["species"].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [57]:
#mapping each species to an integer
species_mapping = {
    'Iris-setosa': 1.0,
    'Iris-versicolor': 2.0,
    'Iris-virginica': 3.0
}

#by using map function we can substitute one value to another
df['species'] = df['species'].map(species_mapping)

In [58]:
df['species'].unique()

array([1., 2., 3.])

In [59]:
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,1.0
1,4.9,3.0,1.4,0.2,1.0
2,4.7,3.2,1.3,0.2,1.0
3,4.6,3.1,1.5,0.2,1.0
4,5.0,3.6,1.4,0.2,1.0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,3.0
146,6.3,2.5,5.0,1.9,3.0
147,6.5,3.0,5.2,2.0,3.0
148,6.2,3.4,5.4,2.3,3.0


In [60]:
#splitting the dataset into X and y

X = df.drop('species', axis=1).values #values is used to convert these dataframes into numpy values
y = df['species'].values

In [74]:
y

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.])

In [61]:
#splitting X and y into train test each
#test_size - 30% and train_size - 70%

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=41) 

In [70]:
#converting X features to FloatTensor
X_train = torch.FloatTensor(X_train) #float tensor because the data values are in float
X_test = torch.FloatTensor(X_test) 

#converting y features to LongTensor
y_train = torch.LongTensor(y_train) #long tensors are 64 bit integers as we set 1.0, 2.0 and 3.0
y_test = torch.LongTensor(y_test)

# TODO: make the mapping of species as integers and compare with the long tensor values

In [71]:
#setting criterion of the model to measure the error

#choosing adam optimizer which goes through in loop and continuously optimizes
#setting a learning rate - if our error does not go low after epoch then we will lower the learning rate
#

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [72]:
#training model

epoch = 100 #determining the number of iterations
losses = []

for i in range(epoch):
    y_pred = model.forward(X_train) #getting predicted results
    loss = criterion(y_pred, y_train) #measuring the loss 
    loss.append(loss.detach().numpy()) #get count of the losses

    if i % 10 == 0: #printing after every 10 epoch
        print("Epoch:", i, "and loss:", loss)

    #back propagating to learn better
    #basically we send error rate of forwards propagation and use it as an input to the network to learn better

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

IndexError: Target 3 is out of bounds.

Braching out

In [76]:
dfx = pd.read_csv('iris-dataset/iris.data', header = None, names = ['sepal_length' , 'sepal_width', 'petal_length', 'petal_width', 'species'])
dfx['species'].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [77]:
dfx['species'] = dfx['species'].replace('Iris-setosa', 0.0)
dfx['species'] = dfx['species'].replace('Iris-virginica', 2.0)
dfx['species'] = dfx['species'].replace('Iris-versicolor', 1.0)

In [80]:
Xi = dfx.drop('species', axis=1).values #values is used to convert these dataframes into numpy values
yi = dfx['species'].values

In [86]:
#splitting X and y into train test each
#test_size - 30% and train_size - 70%

X_trainx, X_testx, y_trainx, y_testx = train_test_split(Xi, yi, test_size=0.3, random_state=41) 

In [87]:
#converting X features to FloatTensor
X_trainx = torch.FloatTensor(X_trainx) #float tensor because the data values are in float
X_testx = torch.FloatTensor(X_testx) 

#converting y features to LongTensor
y_trainx = torch.LongTensor(y_trainx) #long tensors are 64 bit integers as we set 1.0, 2.0 and 3.0
y_testx = torch.LongTensor(y_testx)

In [88]:
#training model

epoch = 100 #determining the number of iterations
losses = []

for i in range(epoch):
    y_predd = model.forward(X_trainx) #getting predicted results
    loss = criterion(y_predd, y_trainx) #measuring the loss 
    losses.append(loss.detach().numpy()) #get count of the losses

    if i % 10 == 0: #printing after every 10 epoch
        print("Epoch:", i, "and loss:", loss)

    #back propagating to learn better
    #basically we send error rate of forwards propagation and use it as an input to the network to learn better

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Epoch: 0 and loss: tensor(0.0192, grad_fn=<NllLossBackward0>)
Epoch: 10 and loss: tensor(0.0174, grad_fn=<NllLossBackward0>)
Epoch: 20 and loss: tensor(0.0157, grad_fn=<NllLossBackward0>)
Epoch: 30 and loss: tensor(0.0142, grad_fn=<NllLossBackward0>)
Epoch: 40 and loss: tensor(0.0130, grad_fn=<NllLossBackward0>)
Epoch: 50 and loss: tensor(0.0120, grad_fn=<NllLossBackward0>)
Epoch: 60 and loss: tensor(0.0112, grad_fn=<NllLossBackward0>)
Epoch: 70 and loss: tensor(0.0105, grad_fn=<NllLossBackward0>)
Epoch: 80 and loss: tensor(0.0099, grad_fn=<NllLossBackward0>)
Epoch: 90 and loss: tensor(0.0093, grad_fn=<NllLossBackward0>)
